In [125]:
!pip install --upgrade pip
!pip3 install tensorflow
!pip3 install torch torchvision

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\giuli\anaconda3\python.exe -m pip install --upgrade pip


In [126]:
import tensorflow as tf
import torchvision
from tensorflow import keras
from tensorflow.keras import layers


import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from astroML.utils import completeness_contamination
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler


import pandas as pd
import numpy as np
import scipy.stats
import pylab as plt
from matplotlib import pyplot as plt
from scipy import stats
import random
import math
import seaborn as sns

In [127]:
import h5py
f = h5py.File('sample_2e7_design_precessing_higherordermodes_3detectors.h5', 'r')
list(f.keys())

['chi1x',
 'chi1y',
 'chi1z',
 'chi2x',
 'chi2y',
 'chi2z',
 'dec',
 'det',
 'iota',
 'mtot',
 'psi',
 'q',
 'ra',
 'snr',
 'z']

In [128]:
d = {i: f[i][:] for i in f.keys()}
data = pd.DataFrame(data=d)
data

,chi1x,chi1y,chi1z,chi2x,chi2y,chi2z,dec,det,iota,mtot,psi,q,ra,snr,z
0,-0.307201,0.276165,-0.430760,-0.161114,-0.092334,-0.414647,-0.329671,0,1.809310,906.803562,1.286031,0.624780,-3.116784,0.822509,2.139659
1,-0.734712,-0.498493,-0.298162,0.057532,-0.844992,0.014708,0.526740,0,0.820820,59.102112,0.934308,0.986650,1.360291,1.093237,2.438707
2,-0.057831,-0.703796,-0.354512,-0.215538,0.016323,0.130543,0.315799,0,2.008206,44.602750,1.535520,0.286199,0.079973,1.305398,1.869461
3,0.015260,-0.202467,-0.492768,-0.237033,-0.365363,-0.876192,0.432105,0,2.018398,534.378032,3.064086,0.538882,-1.780558,9.112974,0.428061
4,-0.016731,0.007848,0.005007,0.147519,0.231879,0.086716,-0.458711,1,2.037363,573.821705,0.752211,0.846310,2.789289,17.525534,0.772609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,-0.192062,-0.132125,0.099840,0.244232,-0.610395,0.205643,0.227502,0,1.712124,47.173930,0.458902,0.721899,2.747641,0.550924,2.942686
19999996,-0.004265,-0.056190,-0.030455,0.080665,-0.263666,-0.028503,-0.289584,0,1.796748,319.515160,2.011788,0.276340,-3.010306,6.303695,1.348554
19999997,-0.558253,-0.107589,0.065246,0.305706,-0.212490,-0.412048,-0.236878,0,0.894741,536.491292,1.776291,0.909982,-1.638381,9.975289,0.826640
19999998,0.084775,-0.042762,0.017719,0.145657,0.474321,0.744677,0.104063,0,1.102068,879.055152,1.317251,0.287587,2.211932,0.920406,3.125594


We rescale our data first. We use only the first 20000 rows. Let's split our dataset in test and train sets.

In [176]:
smaller_data = data[:20000]
smaller_data = smaller_data.drop(['det', 'snr'],axis=1)
smaller_labels = data.det[:20000]

#standardize the data
scaler = StandardScaler()
smaller_data = scaler.fit_transform(smaller_data)

x_train, x_test, y_train, y_test = train_test_split(smaller_data, smaller_labels, train_size = 0.9, random_state=True)

In [177]:
x_train.shape

(18000, 13)

In [178]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(13,1)),
    tf.keras.layers.Conv1D(300, 9, activation="relu",padding='same'),
    tf.keras.layers.MaxPooling1D(3),
    tf.keras.layers.Conv1D(400, 9, activation="relu",padding='same'),
    tf.keras.layers.MaxPooling1D(3),
    tf.keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10)
])

In [179]:
model.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_41 (Conv1D)              │ (None, 13, 300)        │         3,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_41 (MaxPooling1D) │ (None, 4, 300)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_42 (Conv1D)              │ (None, 4, 400)         │     1,080,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_42 (MaxPooling1D) │ (None, 1, 400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 1, 128)         │        51,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 1, 100)         │        12,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 1, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 1, 50)          │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 1, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 1, 10)          │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,153,188 (4.40 MB)

 Trainable params: 1,153,188 (4.40 MB)

 Non-trainable params: 0 (0.00 B)

In [180]:
model.compile(optimizer='nadam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [181]:
model.fit(x_train, y_train, epochs=30, validation_split=0.2)

Epoch 1/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.7833 - loss: 0.7186 - val_accuracy: 0.9344 - val_loss: 0.1630
Epoch 2/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9331 - loss: 0.1646 - val_accuracy: 0.9269 - val_loss: 0.1547
Epoch 3/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9463 - loss: 0.1336 - val_accuracy: 0.9389 - val_loss: 0.1332
Epoch 4/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9557 - loss: 0.1098 - val_accuracy: 0.9342 - val_loss: 0.1324
Epoch 5/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9575 - loss: 0.1060 - val_accuracy: 0.9425 - val_loss: 0.1345
Epoch 6/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9607 - loss: 0.0960 - val_accuracy: 0.9497 - val_loss: 0.1086
Epoch 7/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9636 - loss: 0.0868 - val_accuracy: 0.9544 - val_loss: 0.1077
Epoch 8/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9680 - loss: 0.0812 - val_

In [182]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9746 - loss: 0.0757
Test accuracy:  0.968999981880188
